In [3]:
import pandas as pd
import numpy as np
from this_mapping import supported_genres

tracks_df = pd.read_csv('../edadata.csv')

# Параметры
n_users = 4500
min_interactions = 20
max_interactions = 60
np.random.seed(42)

# Создаём словарь для быстрого доступа к popularity по item_id
item_popularity = tracks_df.set_index('item_id')['popularity'].to_dict()
max_pop = tracks_df['popularity'].max()  # для нормализации
if max_pop == 0:
    max_pop = 1  # избегаем деления на ноль

idx_to_genre = {i: g for i, g in enumerate(supported_genres)}
genre_to_idx = {g: i for i, g in enumerate(supported_genres)}

tracks_df['item_id'] = tracks_df['item_id'].astype(int)

# Декодируем жанр из числа в название
tracks_df['genre_name'] = tracks_df['genre'].map(idx_to_genre)

# Проверка: все ли жанры корректны
if tracks_df['genre_name'].isna().any():
    raise ValueError("Обнаружены некорректные значения жанров в edadata.csv")

# Группируем треки по жанрам
genre_groups = tracks_df.groupby('genre_name')

# Подготовим словарь: для каждого жанра — список item_id и нормализованные веса по popularity
genre_track_pool = {}
for genre, group in genre_groups:
    pop = group['popularity'].values.astype(float)
    if pop.sum() == 0:
        weights = np.ones(len(pop))
    else:
        weights = pop / pop.sum()
    genre_track_pool[genre] = {
        'item_ids': group['item_id'].values,
        'weights': weights
    }

interactions = []

# 1. Выбираем 3–5 любимых жанров с учётом их общей популярности
all_genres = list(genre_track_pool.keys())

# Вычисляем "силу" жанра как сумму популярности его треков
genre_global_pop = {}
for genre in all_genres:
    total_pop = tracks_df[tracks_df['genre_name'] == genre]['popularity'].sum()
    genre_global_pop[genre] = total_pop

bias_weights = np.array([genre_global_pop[g] for g in all_genres], dtype=float)
if bias_weights.sum() == 0:
    bias_weights = np.ones_like(bias_weights)
bias_weights = bias_weights / bias_weights.sum()


for user_id in range(n_users):
    n_fav_genres = np.random.randint(3, 8)
    fav_genres = np.random.choice(all_genres, size=n_fav_genres, replace=False, p=bias_weights)
    
    # Генерируем alpha из экспоненциального распределения — часто даёт 1–2 сильных жанра
    alpha = np.random.exponential(scale=2.0, size=n_fav_genres)
    alpha = np.clip(alpha, 0.1, None)  # избегаем нулей
    genre_prefs = np.random.dirichlet(alpha=alpha)

    # 3. Общее число взаимодействий
    n_interactions = np.random.randint(min_interactions, max_interactions + 1)
    
    # 4. Распределяем по жанрам
    interactions_per_genre = np.random.multinomial(n_interactions, genre_prefs)
    
    user_items = []
    for genre, n_in_genre in zip(fav_genres, interactions_per_genre):
        if n_in_genre == 0:
            continue
        pool = genre_track_pool[genre]
        replace = n_in_genre > len(pool['item_ids'])
        sampled = np.random.choice(
            pool['item_ids'],
            size=n_in_genre,
            replace=replace,
            p=pool['weights']  # ← это popularity внутри жанра
        )
        user_items.extend(sampled)
    
    # Убираем дубликаты
    user_items = list(dict.fromkeys(user_items))[:n_interactions]
    
    # Генерация relevance с учётом popularity трека
    for item_id in user_items:
        pop_val = item_popularity[item_id]
        pop_norm = (pop_val - 62) / (100 - 62)  # нормализуем в [0, 1] относительно твоего диапазона

        # Теперь: чем выше популярность → тем выше шанс "лайка"
        p_like = 0.3 + 0.4 * pop_norm   # [0.3, 0.7]
        p_full = 0.7 - 0.4 * pop_norm   # [0.7, 0.3]

        relevance = np.random.choice([3, 4], p=[p_full, p_like])
        interactions.append({
            'user_id': user_id,
            'item_id': item_id,
            'rating': relevance,
        })

interactions_df = pd.DataFrame(interactions)

# Генерация timestamp
start = pd.Timestamp('2024-10-26')
end = pd.Timestamp('2025-10-26')
time_diff = (end - start).total_seconds()
random_seconds = np.random.rand(len(interactions_df)) * time_diff
interactions_df['timestamp'] = start + pd.to_timedelta(random_seconds, unit='s')
interactions_df['timestamp'] = pd.to_datetime(interactions_df['timestamp'])

# Сохраняем
interactions_df.to_csv("../interaction_data.csv", index=False)